# 0. Libarary 불러오기 및 경로설정

In [1]:
import os
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize
 
from dataset import MaskBaseDataset, BaseAugmentation, MaskSplitByProfileDataset
from datadoubling import DataDoubling
import train as tr

In [2]:
# 테스트 데이터셋 폴더 경로를 지정해주세요.
test_dir = '/opt/ml/input/data/eval'

# 1. Data Path Setting

In [3]:
test_dir = '/opt/ml/input/data/eval' #test데이터 경로
train_dir = '/opt/ml/input/data/train' #train데이터 경로

train_csv = pd.read_csv(os.path.join(train_dir, 'train.csv'))
train_data_images = os.path.join(train_dir, 'images')

print(os.getcwd())
print(train_data_images)

/opt/ml/code
/opt/ml/input/data/train/images


In [4]:
instance = DataDoubling(train_data_images)
instance.copy_dir()

FileExistsError: [Errno 17] File exists: '/opt/ml/input/data/train/images2'

In [5]:
copy_instance = DataDoubling(os.path.join(train_dir, 'images2'))
copy_instance.data_doubling()

doubling completed!


In [6]:
!pip install efficientnet_pytorch

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16445 sha256=3a36ca4f03eb197f41a0a333626138610b27229b787fb2cc44f1d93554e0d262
  Stored in directory: /opt/ml/.cache/pip/wheels/84/b9/90/25a0195cf95fb5533db96f1c77ea3f296b7cc86ae8ae48e3dc
Successfully built efficientnet-pytorch


# 2. Dataset 정의

In [5]:
train_data_images = os.path.join(train_dir, 'images2')
total_dataset = MaskBaseDataset(train_data_images)

inject_transform = BaseAugmentation(224,total_dataset.mean, total_dataset.std )

total_dataset.set_transform(inject_transform)

#print(total_dataset[3])
#print(total_dataset.calc_statistics)

/opt/conda/lib/python3.8/site-packages/torchvision/transforms/transforms.py:280: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


# 3. Data Split to (Train, Valid)

In [6]:
split_object  = MaskSplitByProfileDataset(train_data_images)

inject_transform = BaseAugmentation(224,split_object.mean, split_object.std )
split_object.set_transform(inject_transform)
train_dataset , valid_dataset = split_object.split_dataset()

print('train[1] data : {}, {}'.format(train_dataset[1], len(train_dataset)))
print('valid[1] data : {}, {}'.format(valid_dataset[1], len(valid_dataset)))

train[1] data : (tensor([[[ 1.3777,  1.2288,  1.5597,  ..., -0.5086, -0.5252, -0.5583],
         [ 1.3942,  1.2453,  1.5597,  ..., -0.4094, -0.4259, -0.4425],
         [ 1.4273,  1.2453,  1.5431,  ..., -0.3928, -0.3928, -0.4094],
         ...,
         [ 1.0302,  0.8316,  0.5834,  ..., -1.2036, -1.1540, -1.1043],
         [ 1.2619,  1.2453,  1.2122,  ..., -1.1705, -1.0878, -1.0712],
         [ 0.9475,  1.1129,  1.1957,  ..., -1.1540, -1.0547, -1.0547]],

        [[ 1.2619,  1.1190,  1.4365,  ...,  1.1349,  1.0714,  1.0396],
         [ 1.2778,  1.1349,  1.4365,  ...,  1.2143,  1.1666,  1.1349],
         [ 1.3095,  1.1349,  1.4207,  ...,  1.1984,  1.1825,  1.1666],
         ...,
         [ 1.1825,  0.9920,  0.7856,  ..., -0.9767, -0.9291, -0.8815],
         [ 1.3730,  1.3571,  1.3413,  ..., -0.9450, -0.8656, -0.8497],
         [ 1.0555,  1.1984,  1.2936,  ..., -0.9291, -0.8338, -0.8338]],

        [[ 1.3208,  1.1773,  1.4643,  ...,  0.9701,  0.9223,  0.8904],
         [ 1.3368,  1.1933, 

# 4. Training

In [7]:
#!pip install torchensemble
!python train.py
#print(tr.train(train_data_images, os.path.join(os.getcwd(), 'model.py')))

Namespace(augmentation='CustomAugmentation', batch_size=64, criterion='cross_entropy', data_dir='/opt/ml/input/data/train/images', dataset='MaskSplitByProfileDataset', epochs=21, log_interval=20, lr=6e-05, lr_decay_step=20, model='MyModel', model1='MyModel1', model2='MyModel2', model3='MyModel3', model_dir='./model', name='exp', optimizer='Adam', resize=[224, 224], seed=42, val_ratio=0.2, valid_batch_size=64)
/opt/conda/lib/python3.8/site-packages/torchvision/transforms/transforms.py:280: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
Loaded pretrained weights for efficientnet-b0
Epoch[0/21](20/241) || training loss 7.085 || training accuracy 0.08% || lr 6e-05
Epoch[0/21](40/241) || training loss 6.339 || training accuracy 1.88% || lr 6e-05
Epoch[0/21](60/241) || training loss 5.356 || training accuracy 12.11% || lr 6e-05
Epoch[0/21](80/241) || training loss 4.156 || training accuracy 30.55% |

In [9]:
!python inference.py

Loaded pretrained weights for efficientnet-b0
/opt/conda/lib/python3.8/site-packages/torchvision/transforms/transforms.py:280: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
Calculating inference results..
Inference Done! Inference result saved at ./output/output.csv


In [ ]:
from evaluation import evaluation

print(os.path.join(os.getcwd(), 'output'))
print(test_dir)
#evaluation(test_dir, os.path.join(os.getcwd(), 'output'))